In [1]:
import sys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import getpass

from IPython.display import Markdown
def printmd(string):
    display(Markdown(string))

pd.set_option('display.max_columns', 20) 
pd.set_option('display.max_rows', 1000)


#install Chromedrivermanager
#ChromeDriverManager().install()

### Navigate to web page

In [719]:
driver = webdriver.Chrome(executable_path='path of chromedriver')

#Open login page
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

# #Enter login info:
elementID = driver.find_element_by_id('username')
elementID.send_keys('bahuang23@gmail.com')

elementID = driver.find_element_by_id('password')
elementID.send_keys(getpass.getpass("Password: "))
#Note: replace the keys "username" and "password" with your LinkedIn login info

elementID.submit()


### Webpages

In [686]:
DA_url_past_wk = 'https://www.linkedin.com/jobs/search/?f_E=2%2C3&f_TPR=r604800&geoId=90000070&keywords=data%20analyst&location=New%20York%20City%20Metropolitan%20Area'
DA_url_past_mo = 'https://www.linkedin.com/jobs/search/?f_E=2%2C3&f_TPR=r2592000&geoId=90000070&keywords=data%20analyst&location=New%20York%20City%20Metropolitan%20Area'
BA_url_past_mo = 'https://www.linkedin.com/jobs/search/?f_E=2%2C3&f_TPR=r2592000&geoId=90000070&keywords=business%20analyst&location=New%20York%20City%20Metropolitan%20Area'
rec_jobs = 'https://www.linkedin.com/jobs/search/?currentJobId=2536632686&pivotType=jymbii'

### Go to url

In [782]:
driver.get(BA_url_past_mo)

### Main script

In [783]:
job_desc_details_list = []

In [784]:
for page in range(2,5):    
    
    #Scrolling to load elements
    for i in range(4):
        driver.execute_script("window.document.getElementsByClassName('jobs-search-results')[0].scrollBy(0, document.getElementsByClassName('jobs-search-results')[0].scrollHeight/4)") 
        time.sleep(0.5)

    for _ in range(2):
        for _ in range(2):
            driver.execute_script("window.document.getElementsByClassName('jobs-search-results')[0].scrollBy(0, document.getElementsByClassName('jobs-search-results')[0].scrollHeight/4)")
            time.sleep(0.5)
        for _ in range(2):
            driver.execute_script("window.document.getElementsByClassName('jobs-search-results')[0].scrollBy(0, -document.getElementsByClassName('jobs-search-results')[0].scrollHeight/4)")
            time.sleep(0.5)

    
    #Get web source
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    
    #All jobs
    tiles = soup.find_all('li', class_ = "jobs-search-results__list-item occludable-update p0 relative ember-view")

    print("# of listings after scrolling:", len(tiles))



    
    #Extracting
    for i in tiles:
        id = i.get('id')
        driver.find_element_by_id(id).click()
        time.sleep(0.5)
        
        

        job_desc_details = []
        
        
        #Job web source 
        html_source = driver.page_source

        soup = BeautifulSoup(html_source, 'html.parser')
        
        
        #Company name
        try:
            company_name = soup.find("a", class_="jobs-details-top-card__company-url t-black--light t-normal ember-view").getText().strip()
        except:
            if soup.find("div", class_="job-card-container__company-name"):
                company_name = soup.find("div", class_="job-card-container__company-name").getText().strip()
            elif not soup.find("div", class_="job-card-container__company-name"):
                company_name = 'No company name'


        job_desc_details.append(company_name)
        
        
        #Title
        job_title = soup.find("h2", class_="jobs-details-top-card__job-title t-20 t-black t-normal").getText().strip()
        job_desc_details.append(job_title)
        
        
        #Location
        try: 
            location = soup.find("a", class_="jobs-details-top-card__exact-location t-black--light link-without-visited-state").getText().strip()
        except:
            location = soup.find("span", class_="jobs-details-top-card__bullet").getText().strip()

        job_desc_details.append(location)
        
        
        #Views, Posted date
        views_list = [i for i in soup.find_all('p', class_="t-12 t-black--light mt2 mb0")[0].stripped_strings]
        if views_list[0] == 'New':
            views = views_list[4]
            posted_date = views_list[2]
        else:
            try:
                views = views_list[3]
                posted_date = views_list[1]
            except:
                views = "Couldn't get views"
                posted_date = views_list[1]

        job_desc_details.append(views)
        job_desc_details.append(posted_date)

        
        
        #Applicants, Level, Company size, Industry
        job_details_short = soup.find_all("span", class_="jobs-details-job-summary__text--ellipsis")
        
        applicants_no = job_details_short[0].getText().strip()
        level = job_details_short[1].getText().strip()
        try:
            company_size = job_details_short[2].getText().strip()
            industry = job_details_short[3].getText().strip()
        except:
            company_size = 'Company page not claimed'
            industry = 'NA'
        
        
        
        job_desc_details.extend([applicants_no, level, company_size, industry])
        
        
        #Connections
        
        connections_element = soup.find_all("li", class_="artdeco-list__item jobs-details-job-summary__section")[1].find_all("div", class_="job-flavors__label t-12 t-black--light t-normal")
        no_connections_element = soup.find_all("div", class_="mb1")
        
        if connections_element:
            connections = []
            for i in connections_element:
                connections.append(i.getText().strip())
            job_desc_details.append(connections)
        else:
            job_desc_details.append(no_connections_element[0].getText().strip())
        
        
        #JD
        job_description = soup.find("div", class_="jobs-box__html-content jobs-description-content__text t-14 t-normal").getText().strip()

        job_desc_details.append(job_description)
        
        
        #URL
        job_desc_details.append(driver.current_url)
        
        
        #Adding all job info to list
        job_desc_details_list.append(job_desc_details)
    
    
    #Click next job
    driver.find_element_by_xpath(f"//ul[@class='artdeco-pagination__pages artdeco-pagination__pages--number']/li[@data-test-pagination-page-btn = '{page}']").click()
    time.sleep(2)


# of listings after scrolling: 25
# of listings after scrolling: 25
# of listings after scrolling: 25


### Putting jobs info into dataframe

In [800]:
job_desc_details_list_df = pd.DataFrame(job_desc_details_list, columns = ['Company Name', 'Job Title', 'Location', 'Views', 'Posted_date', '# of applicants', 'Level', 'Company size', 'Industry', 
                                                                          'Connections', 'Job Description', 'url'])

In [812]:
job_desc_details_list_df.head(10)

,Company Name,Job Title,Location,Views,Posted_date,# of applicants,Level,Company size,Industry,Connections,Job Description,url
0,Greenlots,Business Development Analyst - East Coast,"New York, NY",115 views,Posted 3 weeks ago,12 applicants,Associate,51-200 employees,Automotive,You have 0 connections at this company.,Business Development Analyst – East Coast\n\nJ...,https://www.linkedin.com/jobs/search/?currentJ...
1,"Ross Stores, Inc.","Business Analyst, Business Information","New York, NY","1,332 views",Posted 3 weeks ago,600 applicants,Associate,10001+ employees,Retail,[2 alumni],Posted by\n \n\n\n\n\n\n\n\n\n D...,https://www.linkedin.com/jobs/search/?currentJ...
2,TEKsystems,Business Systems Analyst,"New York, NY",33 views,Posted 1 week ago,5 applicants,Associate,"1,001-5,000 employees",Information Technology & Services,[8 alumni],Summary\n\nThe main function of a GBS business...,https://www.linkedin.com/jobs/search/?currentJ...
3,WeWork,"Associate, Data Analyst","New York, NY",597 views,Posted 14 hours ago,171 applicants,Associate,"5,001-10,000 employees",Real Estate,[11 alumni],WeWork Mission\n\nWeWork is the platform for c...,https://www.linkedin.com/jobs/search/?currentJ...
4,Peloton Interactive,"Data/Reporting Analyst, IT Service Design","New York, NY",275 views,Posted 1 day ago,45 applicants,Associate,"1,001-5,000 employees","Health, Wellness & Fitness","[3 company alumni, 19 alumni]",Peloton’s IT organization is seeking an experi...,https://www.linkedin.com/jobs/search/?currentJ...
5,Etsy,Business Development Analyst,"Brooklyn, NY",266 views,Posted 1 week ago,46 applicants,Associate,"1,001-5,000 employees",Internet,"[1 connection, 2 company alumni]",Company Description\n\nEtsy is the global mark...,https://www.linkedin.com/jobs/search/?currentJ...
6,Feather,Business Intelligence Analyst,"New York, NY",326 views,Posted 2 weeks ago,83 applicants,Associate,51-200 employees,Furniture,You have 0 connections at this company.,"At Feather, our mission is to transform people...",https://www.linkedin.com/jobs/search/?currentJ...
7,"Glossier, Inc.",Program Analyst,"New York, NY",457 views,Posted 4 days ago,26 applicants,Associate,201-500 employees,Consumer Goods,[1 alum],Overview\n\nThe Programs Business Analyst II w...,https://www.linkedin.com/jobs/search/?currentJ...
8,Nomura,Data Management Analyst,New York City Metropolitan Area,517 views,Posted 1 day ago,239 applicants,Associate,10001+ employees,Financial Services,"[1 connection, 19 alumni]",Company overview\nNomura is an Asia-based fina...,https://www.linkedin.com/jobs/search/?currentJ...
9,WebMD,Associate Business/Data Analyst,"New York, NY",449 views,Posted 2 weeks ago,108 applicants,Associate,"1,001-5,000 employees",Internet,[4 alumni],Jobson Healthcare Information (JHI) is a premi...,https://www.linkedin.com/jobs/search/?currentJ...


### Filter jobs

In [805]:
#Specify words to remove
words_to_remove = ['4+ years', '5+ years', '5 years', '6 years', '10+ years', '10 years', '6+ years', 'Five or more years', 'Six or more years', 'Three or more years', 'four or more years',
                'Five plus years', 'five or more years', 'six or more years', 'Four or more years']


consider_indexes = []

for i in range(len(job_desc_details_list_df)):
    jd = job_desc_details_list_df.loc[i, 'Job Description']
    if any(x in jd for x in words_to_remove):
        consider_indexes.append(i)
        
consider_jobs = job_desc_details_list_df[~job_desc_details_list_df.index.isin(consider_indexes)]

In [813]:
#Filtered
consider_jobs.head(10)

,Company Name,Job Title,Location,Views,Posted_date,# of applicants,Level,Company size,Industry,Connections,Job Description,url
2,TEKsystems,Business Systems Analyst,"New York, NY",33 views,Posted 1 week ago,5 applicants,Associate,"1,001-5,000 employees",Information Technology & Services,[8 alumni],Summary\n\nThe main function of a GBS business...,https://www.linkedin.com/jobs/search/?currentJ...
6,Feather,Business Intelligence Analyst,"New York, NY",326 views,Posted 2 weeks ago,83 applicants,Associate,51-200 employees,Furniture,You have 0 connections at this company.,"At Feather, our mission is to transform people...",https://www.linkedin.com/jobs/search/?currentJ...
9,WebMD,Associate Business/Data Analyst,"New York, NY",449 views,Posted 2 weeks ago,108 applicants,Associate,"1,001-5,000 employees",Internet,[4 alumni],Jobson Healthcare Information (JHI) is a premi...,https://www.linkedin.com/jobs/search/?currentJ...
10,Kaia Health,"Data Analyst, Business","New York, NY",264 views,Posted 1 week ago,37 applicants,Associate,51-200 employees,Computer Software,You have 0 connections at this company.,Kaia is a mission-focused health technology co...,https://www.linkedin.com/jobs/search/?currentJ...
11,Enfusion,Strategy Business Analyst,"New York, NY",362 views,Posted 3 weeks ago,143 applicants,Associate,"501-1,000 employees",Financial Services,[3 alumni],Enfusion is a pioneer in developing innovative...,https://www.linkedin.com/jobs/search/?currentJ...
12,Healthfirst,Business Data Analyst,"New York, NY",61 views,Posted 4 weeks ago,6 applicants,Associate,"1,001-5,000 employees",Hospital & Health Care,[2 alumni],The Business Data Analyst (BDA) is responsible...,https://www.linkedin.com/jobs/search/?currentJ...
13,Expert Institute,Business Analyst,"New York, NY",525 views,Posted 2 weeks ago,233 applicants,Associate,51-200 employees,Information Technology & Services,You have 0 connections at this company.,"ABOUT US:At Expert Institute, we’re building t...",https://www.linkedin.com/jobs/search/?currentJ...
14,Wiley,Business Analyst,"Hoboken, NJ",74 views,Posted 1 week ago,11 applicants,Associate,"1,001-5,000 employees",Publishing,[5 alumni],"At Wiley, we welcome you for who you are, the ...",https://www.linkedin.com/jobs/search/?currentJ...
18,Industrious,Business Analyst,"New York, NY","1,073 views",Posted 4 days ago,151 applicants,Associate,201-500 employees,Hospitality,[2 alumni],About The Company\n\nIndustrious’ private offi...,https://www.linkedin.com/jobs/search/?currentJ...
20,Sony Electronics,Associate Data Analyst,"Paramus, NJ",189 views,Posted 4 weeks ago,37 applicants,Associate,10001+ employees,Consumer Electronics,[4 alumni],"Sony Electronics, Inc. look for the risk-taker...",https://www.linkedin.com/jobs/search/?currentJ...


In [814]:
for i in range(len(consider_jobs[:5])):
    
    company_name = consider_jobs.iloc[i, 0]
    job_title = consider_jobs.iloc[i, 1]
    location = consider_jobs.iloc[i, 2]
    views = consider_jobs.iloc[i, 3]
    posted_date = consider_jobs.iloc[i, 4]
    no_applicants = consider_jobs.iloc[i, 5]
    level = consider_jobs.iloc[i, 6]
    company_size = consider_jobs.iloc[i, 7]
    industry = consider_jobs.iloc[i, 8]
    connections = ','.join(consider_jobs.iloc[i, 9])
    job_description = consider_jobs.iloc[i, 10]
    url = consider_jobs.iloc[i, 11]
    
    printmd('**'+ company_name +'**')
    printmd('**'+ job_title +'**')
    printmd('**'+ location +'**')
    printmd('**'+ views +'**')
    printmd('**'+ posted_date +'**')
    printmd('**'+ no_applicants +'**')
    printmd('**'+ level +'**')
    printmd('**'+ company_size +'**')
    printmd('**'+ industry +'**')
    printmd('**'+ connections +'**')
    printmd('**'+ job_description +'**')
    print('URL:', url)
    printmd('*'*500)
    printmd('\n')

**TEKsystems**

**Business Systems Analyst**

**New York, NY**

**33 views**

**Posted 1 week ago**

...

**Financial Services**

**3 alumni**

**Enfusion is a pioneer in developing innovative, native cloud investment management software, analytics, and managed services for asset managers and hedge funds around the globe. We underpin the investment operations of more than 500 fund managers from our nine global offices spanning four continents. Our unique, seamless, SaaS-based platform is reshaping the global investment operations landscape by removing traditional information boundaries and uniting the front-, middle- and back-office into one system, with one unified data set. Passionate about client service, we complement our software solutions with expert middle- and back-office managed services. In fact, we were recently recognized as Best Managed Services Provider at the 2020 HFM EU Services Awards.

Just as Enfusion is committed to excellence in our solutions and services, we’re absolutely committed to the growth, development, and well-being of our people. At Enfusion, you will help fund managers around the world streamline their operations, mitigate risk, and improve transparency and reporting so they can confidently focus on what they do best – invest. Similarly, Enfusion is your opportunity to pursue your passion, showcase and further develop your skills, and launch a meaningful and rewarding career. Whether you are developing our industry-leading technology, delivering world-class client service, creating leading edge analytics, managing critical client operations, winning new business, or supporting Enfusion's ever-expanding global footprint, you will be part of a dynamic, fast-growing, and innovative company committed to your success. Our environment is inclusive, collaborative, and entrepreneurial. We value diversity, foster and reward creative exploration, and strive to continually exceed client expectations.

Learn more about your career at Enfusion by visiting www.enfusion.com

The Strategy Business Analyst role will report to our Head of Strategy directly, help support and create internal and external strategies for Enfusion.

What You'll Do
 Business Analysts help create scalable solutions for our clients, by identifying opportunities and strategies to enhance the product, service delivery and a client efficiency. Identify strategies for growth and value creation Develop the appropriate business models, operating models, and capabilities to support their strategic vision Maximize the ROI on technology investments and leverage technology and Cloud trends to architect future business strategies Lead data analysis efforts to evaluate operating performance, drive external and internal reporting, and augment organic growth and operating expense planning Support internal market intelligence, including researching industry trends, identifying growth opportunities, and assessing competitive landscapes 

Who You Are
 Minimum of a Bachelor’s degree and 2+ years consulting/and or industry experience Superior problem-solving skills and structured thinking Experience in Financial Services sector (e.g., banking, insurance, payment), FinTech, and/or knowledge with digital financial products and platforms preferred Proven track record of delivering results Both technical and soft skills to deliver a complex program Strong verbal and written communication that can be tailored for a specific audience 

Enfusion offers a competitive compensation and benefits package including medical, dental, and vision coverage (Enfusion covers 100% of the premiums for employees, spouses, domestics partners and qualified dependents); a 401(k) plan with a employer match; a generous Paid Time Off policy; and paid parental leave. Other perks include commuter benefits, an employee assistance program, and a Flexible Spending Account (FSA) program. All benefits and programs are subject to applicable eligibility requirements.

Enfusion is fully committed to providing equal employment opportunity to job applicants and employees in recruitment, hiring, employment, compensation, benefits, promotions, transfers, training, and all other terms and conditions of employment. Enfusion will not discriminate on the basis of age, race, color, gender, marital status, sexual orientation, gender identity, pregnancy, national origin, religion, veteran status, physical or mental disability, genetic information, creed, citizenship or any other status protected by laws or regulations in the locations where we do business. We endeavor to maintain a drug-free workplace.
 

 

          Seniority Level
        

          Associate
        




          Industry
        


              Information Technology & Services
            

              Computer Software
            

              Financial Services
            




        Employment Type
      

        Full-time
      





          Job Functions
        


              Research
            

              Analyst
            

              Information Technology**

URL: https://www.linkedin.com/jobs/search/?currentJobId=2487480565&f_E=2%2C3&f_TPR=r2592000&geoId=90000070&keywords=business%20analyst&location=New%20York%20City%20Metropolitan%20Area


********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************